In [22]:
import os, sys, timeit

import numpy as np
from numpy.lib.stride_tricks import as_strided
import scipy as sp
from scipy.ndimage import convolve
import PIL as pillow
from PIL import Image

import scipy.linalg
%matplotlib inline 
import matplotlib.pyplot as plt

In [31]:
x = 3000
y = 3000
output_buffer = np.empty((x, y, 3), dtype=np.float64)
output_buffer = np.random.rand(output_buffer.shape[0], output_buffer.shape[1], 3)

size = (300,300)

# numpy.array(image.Image)

small = Downsample(output_buffer)
tic=timeit.default_timer()
small.one(size)
toc=timeit.default_timer()
del small
print("one: ", toc - tic,  "sec \n")

small = Downsample(output_buffer)
tic=timeit.default_timer()
small.two(size)
toc=timeit.default_timer()
del small
print("two: ", toc - tic,  "sec \n")

small = Downsample(output_buffer)
tic=timeit.default_timer()
small.three(size)
toc=timeit.default_timer()
del small
print("three: ", toc - tic,  "sec \n")


small = Downsample(output_buffer)
tic=timeit.default_timer()
small.five(size)
toc=timeit.default_timer()
del small
print("five: ", toc - tic,  "sec \n")


small = Downsample(output_buffer)
tic=timeit.default_timer()
small.eight(size)
toc=timeit.default_timer()
del small
print("eight: ", toc - tic,  "sec \n")


small = Downsample(output_buffer)
tic=timeit.default_timer()
small.one(size)
toc=timeit.default_timer()
del small
print("ten: ", toc - tic,  "sec \n")


small = Downsample(output_buffer)
tic=timeit.default_timer()
img = Image.fromarray(small.image, mode="RGB")
once = np.array(img)
toc=timeit.default_timer()
del small
print("From array to image and back: ", toc - tic,  "sec \n")


one:  0.052073577000555815 sec 

two:  0.10741131999930076 sec 

three:  0.0325151030010602 sec 

five:  0.017025675000695628 sec 

eight:  1.9185482150005555 sec 

ten:  0.052681621000374435 sec 

From array to image and back:  0.12454987000091933 sec 



In [17]:
size = 128, 128

class Downsample:
    
    def __init__(self, image):
        self.image = image
    
    def zero(self, size): # only works for single channel images
        bin_fac = 1
        assert( len( img.shape ) == 2 )
        imgX0 = img.shape[0] # slow axis
        imgX1 = img.shape[1] # fast axis

        r0 = imgX0 % bin_fac
        r1 = imgX1 % bin_fac

        img = np.copy( img) [:imgX0 - r0, :imgX1-r1]
        # bin the fast  axis 
        img_C = img.ravel(order='C')
        img = np.average( [ img_C[i::bin_fac] for i in xrange( bin_fac )   ],axis=0)
        img = img.reshape( (imgX0-r0, (imgX1-r1)/bin_fac ) , order='C')

        # bin the slow axis
        img_F = img.ravel(order='F')
        img = np.average( [ img_F[i::bin_fac] for i in xrange( bin_fac )   ],axis=0)
        img = img.reshape( ((imgX0-r0)/bin_fac, (imgX1-r1)/bin_fac ), order='F' )
        self.reduced_image = img
        
    def one(self, size): # tested
        im = Image.fromarray(self.image, mode="RGB")
        im.thumbnail(size, Image.ANTIALIAS)
        self.reduced_image = im
        
        # im.save(outfile, "JPEG")
        
    # almost the same as #4
    def two(self, size): # tested
        basewidth = size[0]
        img = Image.fromarray(self.image, mode="RGB")
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth, hsize), Image.ANTIALIAS)
        self.reduced_image = img
        
        # im.save(outfile, "JPEG")
    
    def three(self, size): # tested
        #  Image.NEAREST, Image.BOX, Image.BILINEAR, Image.HAMMING, Image.BICUBIC or Image.LANCZOS
        img = Image.fromarray(self.image, mode="RGB")
        img = img.resize(size, resample=Image.ANTIALIAS, reducing_gap=1)
        self.reduced_image = img
    
    def four(self, size): # don't understand
        bin_fac = size[0]
        strided = as_strided(self.image,
                             shape=(self.image.shape[0]//bin_fac, self.image.shape[1]//bin_fac, bin_fac, bin_fac),
                             strides=((self.image.strides[0]*bin_fac, self.image.strides[1]*bin_fac)+self.image.strides))
        return strided.mean(axis=-1).mean(axis=-1)  # order is NOT important! See notes..
    
    def five(self, size): # tested, not sure if either is better
        h = size[0] / self.image.shape[0]
        w = size[1] / self.image.shape[1]
        img = scipy.ndimage.zoom(self.image, (h, w, 1), order=0)
        self.reduced_image = Image.fromarray(img, mode="RGB")
                
    def eight(self, size): # tested
        h = size[0] / self.image.shape[0]
        w = size[1] / self.image.shape[1]
        img = scipy.ndimage.interpolation.zoom(self.image, (h, w, 1.0))
        self.reduced_image = Image.fromarray(img, mode="RGB")
           
    def ten(self, size): # tested
        h = self.image.shape[0] / size[0]
        w = self.image.shape[1] / size[1]
        self.reduced_image = Image.fromarray(self.image[0::int(h),0::int(w)], mode="RGB")